In [2]:
import numpy as np
#from scipy import linalg as LA

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import matplotlib.pyplot as plt

import h5py
import pickle


import fastjet as fj
import Nsubjettiness as Nsub
import numpy as np
import pandas as pd

import glob as glob
import time

In [10]:
1+1

2

In [6]:
# files = ['/home/jack/projects/EMD_VAE/compression_project/data/B_background/B_background_' + str(i) + '.dat' for i in range(1,41)]
# out_fn = '/home/jack/projects/EMD_VAE/compression_project/data/B_background.h5'

files = ['/home/jack/projects/EMD_VAE/compression_project/data/B_signal/B_signal_1_1_1.dat']
out_fn = '/home/jack/projects/EMD_VAE/compression_project/data/B_signal.h5'

# files = ['/home/jack/projects/EMD_VAE/compression_project/data/h_signal/h_signal_1_1_1.dat']
# out_fn = '/home/jack/projects/EMD_VAE/compression_project/data/h_signal.h5'

# files = ['/home/jack/projects/EMD_VAE/compression_project/data/hv_signal/hv_signal_1_1_1.dat']
# out_fn = '/home/jack/projects/EMD_VAE/compression_project/data/hv_signal.h5'



# path to file
# fn =  '/home/jack/projects/EMD_VAE/compression_project/data/train.h5'
# out_fn = '/home/jack/projects/EMD_VAE/compression_project/data/train_processed.h5'
# fn =  '/media/jcollins/MAGIC!/monoW-data-3.h5'

# Option 1: Load everything into memory
# df = pd.read_hdf(fn,key='table',stop=10000000)
# print(df.shape)
# print("Memory in GB:",sum(df.memory_usage(deep=True)) / (1024**3)+sum(df.memory_usage(deep=True)) / (1024**3))

In [8]:
np.readtext(files[0])

AttributeError: module 'numpy' has no attribute 'readtext'

In [3]:
data = df.values[:,:800].reshape(-1,200,4)
labels = df.values[:,-1]

In [4]:
chunksize = 10000
numchunks = int(len(data)/chunksize)

columns = ['is_top']
for k in range(50):
    columns += ['pT' + str(k), 'eta' + str(k), 'phi' + str(k), 'E' + str(k)]

start = time.time()

for k in range(numchunks):
    print("Processing chunk", k, "of", numchunks)
    raw_data = data[k*chunksize:(k+1)*chunksize]
    
  
    chunk = np.zeros((chunksize,50,4))

    jet_def_small = fj.JetDefinition(fj.kt_algorithm, 1.0)

    for i in range(chunksize):    
        
        event = raw_data[i]
        
        particles = []
        for particle in event:
            particles += [fj.PseudoJet(particle[1],particle[2],particle[3],particle[0])]

        sequence_small = fj.ClusterSequence(particles,jet_def_small)
        jets = fj.sorted_by_pt(sequence_small.exclusive_jets(min(50,len(particles))))

        for j, jet in enumerate(jets):
            chunk[i,j,0] = jet.pt()
            chunk[i,j,1] = jet.eta()
            chunk[i,j,2] = jet.phi()
            chunk[i,j,3] = jet.e()
            

    outdata = chunk.reshape((-1,200))

    end = time.time()
    print("Time:", (end-start)/60, "minutes")
    
    outstore = pd.HDFStore(out_fn)
    dictionary = dict(zip(columns, np.concatenate((labels[k*chunksize:(k+1)*chunksize].reshape((-1,1)),outdata),axis=1).T))
    out_df = pd.DataFrame(dictionary,columns=columns)
    outstore.append('particles',out_df)
    outstore.close()

Processing chunk 0 of 121
Time: 0.43899981180826825 minutes
Processing chunk 1 of 121
Time: 0.887018636862437 minutes
Processing chunk 2 of 121
Time: 1.335149097442627 minutes
Processing chunk 3 of 121
Time: 1.7929732879002889 minutes
Processing chunk 4 of 121
Time: 2.253246625264486 minutes
Processing chunk 5 of 121
Time: 2.7099164883295694 minutes
Processing chunk 6 of 121
Time: 3.152339545885722 minutes
Processing chunk 7 of 121
Time: 3.6012298027674357 minutes
Processing chunk 8 of 121
Time: 4.044631103674571 minutes
Processing chunk 9 of 121
Time: 4.492407993475596 minutes
Processing chunk 10 of 121
Time: 4.94218133687973 minutes
Processing chunk 11 of 121
Time: 5.394102847576141 minutes
Processing chunk 12 of 121
Time: 5.848935214678447 minutes
Processing chunk 13 of 121
Time: 6.30041842063268 minutes
Processing chunk 14 of 121
Time: 6.748877747853597 minutes
Processing chunk 15 of 121
Time: 7.194232316811879 minutes
Processing chunk 16 of 121
Time: 7.64552313486735 minutes
Proce

In [22]:
out_df

,is_top,pT0,eta0,phi0,E0,pT1,eta1,phi1,E1,pT2,...,phi47,E47,pT48,eta48,phi48,E48,pT49,eta49,phi49,E49
0,1.0,51.849997,1.768332,5.609193,156.371521,49.611246,1.994141,5.027391,185.596207,34.990978,...,5.686694,2.041802,0.604068,2.197542,5.679226,2.752714,0.597195,1.493743,5.243583,1.396917
1,0.0,159.724438,0.081100,4.258998,160.250000,122.269950,0.085748,4.292995,122.719734,82.160128,...,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000
2,1.0,217.000082,-0.311597,0.481409,227.620178,32.760366,-0.616653,0.454944,39.189776,30.458613,...,0.509920,0.746174,0.490080,-0.825150,0.454292,0.666605,0.488206,-0.163133,0.101405,0.494716
3,0.0,196.316531,-1.250986,4.141562,371.039062,154.821241,-1.235963,4.177301,288.915588,55.921421,...,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000
4,1.0,96.252062,-1.195398,1.781221,173.612518,75.236005,-1.551132,1.696583,185.412140,61.420868,...,1.907763,1.692546,0.360403,-1.428343,1.992111,0.794956,0.312882,-1.261499,2.361450,0.596657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,258.883963,-0.902603,3.192828,371.696014,79.498192,-0.965936,3.216140,119.560165,71.398762,...,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000
9996,0.0,246.463206,0.326021,2.974069,259.677856,110.312200,0.335045,2.979895,116.561905,100.166834,...,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000
9997,0.0,272.433751,-0.216418,5.202197,278.838623,153.455992,-0.236826,5.181745,157.779541,54.021202,...,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000
9998,1.0,113.931559,-1.886180,5.089077,384.278717,64.617423,-1.548094,5.266336,158.802155,47.227176,...,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000,0.000000,100000.000000,0.000000,0.000000


In [19]:
labels[:20]

array([0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 1.])

In [40]:
chunk[passlist].reshape((-1,150)).shape

(2, 150)

In [31]:
chunk.shape

(50000, 50, 3)

In [38]:
numpass

1

In [32]:
passlist.shape

(50000,)

In [42]:
passlist[:10]

array([ True, False, False, False,  True, False, False, False, False,
       False])

In [33]:
outdata.shape

(1, 150)

In [23]:
sorted_pmuons

array([[  0.   , -13.   ,   7.296,  -0.858,   2.203,  10.151],
       [  0.   , -13.   ,   6.446,  -1.955,   1.532,  23.22 ],
       [  0.   , -13.   ,   6.229,  -0.883,   2.148,   8.821],
       [  0.   , -13.   ,   5.495,  -0.809,   2.177,   7.393],
       [  0.   , -13.   ,   5.429,  -0.828,   2.15 ,   7.401],
       [  0.   , -13.   ,   2.928,  -1.939,   1.475,  10.391],
       [  0.   , -13.   ,   1.266,  -1.812,   1.354,   3.981],
       [  0.   , -13.   ,   1.04 ,  -0.656,   1.925,   1.276],
       [  0.   , -13.   ,   0.781,  -1.075,   1.781,   1.282],
       [  0.   , -13.   ,   0.43 ,  -1.914,   2.016,   1.492],
       [  0.   , -13.   ,   0.258,  -1.535,   1.006,   0.635],
       [  0.   , -13.   ,   0.096,  -1.12 ,   2.478,   0.194]])

In [7]:
chunksize = 10000
numchunks = int(len(data)/chunksize)

columns = []
for i in range(50):
    columns += ['pT' + str(i), 'eta' + str(i), 'phi' + str(i)]

start = time.time()

for i in range(numchunks):
    print("Loading", file)
    raw_data = data[i*chunksize,(i+1)*chunksize]
    
    #eta cut
    raw_data = raw_data[np.abs(raw_data[:,3])<5]
    
    
    chunk = np.zeros((numevents,50,3))


    jet_def_small = fj.JetDefinition(fj.kt_algorithm, 1.0)

    for i in range(numevents):

        if i%10000 == 0:
            print("Running event", i, "of", numevents,". Mins elapsed:", (time.time()-start)/60)

        event = raw_data[raw_data[:,0]==i]
        PIDs = event[:,1]
        ismuon = PIDS
        
        
        particles = []

        for particle in event:
            particles += [fj.PtYPhiM(particle[2],particle[3],particle[4],0)]

        sequence_small = fj.ClusterSequence(particles,jet_def_small)
        jets = fj.sorted_by_pt(sequence_small.exclusive_jets(min(50,len(particles))))

        for j, jet in enumerate(jets):
            chunk[i,j,0] = jet.pt()
            chunk[i,j,1] = jet.eta()
            chunk[i,j,2] = jet.phi()

            
    outdata = chunk.reshape((-1,150))

    end = time.time()
    print("Time:", (end-start)/60, "minutes")
    
    outstore = pd.HDFStore('/home/jack/projects/EMD_VAE/compression_project/data/hv_signal.h5')
    dictionary = dict(zip(columns, outdata.T))
    out_df = pd.DataFrame(dictionary,columns=columns)
    outstore.append('particles',out_df)
    outstore.close()

Loading /home/jack/projects/EMD_VAE/compression_project/data/hv_signal/hv_signal_1_1_1.dat
Running event 0 of 50000 . Mins elapsed: 3.0554272294044496
Running event 10000 of 50000 . Mins elapsed: 17.178145321210227
Running event 20000 of 50000 . Mins elapsed: 31.283803610006967
Running event 30000 of 50000 . Mins elapsed: 45.38832330703735
Running event 40000 of 50000 . Mins elapsed: 542.5198963681856
Time: 558.7479121009509 minutes


In [15]:
fn='/home/jack/projects/EMD_VAE/compression_project/data/B_background.h5'
df = pd.read_hdf(fn,stop=10000000)
print(df.shape)
print("Memory in GB:",sum(df.memory_usage(deep=True)) / (1024**3)+sum(df.memory_usage(deep=True)) / (1024**3))
data = df.values.reshape((-1,50,4))
num=0
for i in range(len(data)):
    if np.all(data[i] == data[0]):
        num = num + 1
print("Number of repeats:",num)

(2000000, 150)
Memory in GB: 4.500150680541992
Number of repeats: 1


In [12]:
)


1
